# Camera Calibration by Zhang's Method
https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr98-71.pdf

<t>Given  : World Coordinates, Pixel Coordinates</t>

<t>Output : Intrinsic Matrix, Extrinsic Matrix</t>

In [71]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import glob

## Reading Images

In [72]:
files = glob.glob('../data/undistorted/*.png')
imgs = []
for f in files:
    img = cv2.imread(f)
    imgs.append(img)


## Create Pattern to Detect Chessboard and Get the World Coordinates

In [73]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
chessSize = (24,17)
squareSize = 0.015

# object points
objp = np.zeros((chessSize[0]*chessSize[1],2),np.float32)
objp[:,:2] = np.mgrid[0:chessSize[0],0:chessSize[1]].T.reshape(-1,2)
objp = objp * squareSize

In [74]:
def getImagesPoints(imgs, chessSize):
    # Store all the object points and image points from all the images
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d point in the image plane
    for img in imgs:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, chessSize, None)
        if ret == True:
            objpoints.append(objp)
            corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
            corners = corners.reshape(-1,2)
            imgpoints.append(corners)

    imgpoints, objpoints = np.array(imgpoints), np.array(objpoints)
    return imgpoints, objpoints

In [75]:
def displayCorners(imgs, imgpoints, chessSize, save_dir):
    for i, img in enumerate(imgs):
        corners = imgpoints[i]
        cv2.drawChessboardCorners(img, chessSize, corners, True)
        
        fname = os.path.join(save_dir,str(i+1) + ".png")
        cv2.imwrite(fname, img)

In [76]:
imgpoints, objpoints = getImagesPoints(imgs, chessSize)

In [77]:
displayCorners(imgs, imgpoints, chessSize, '../data/corners')

## Prepare Utils
$$
s \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = K \begin{bmatrix} r_1 & r_2 & r_3 & t\end{bmatrix} \begin{bmatrix} X \\ Y \\ Z \\ 1\end{bmatrix}
$$

We assume the model plane is on Z = 0 of the world coordinate system (chessboard)
$$
s \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = K \begin{bmatrix} r_1 & r_2 & t\end{bmatrix} \begin{bmatrix} X \\ Y \\ 1\end{bmatrix} (1)
$$
The 3x3 matrix H is defined only up to a scale factor

Expand (1) as linear equation and Rearrange terms, we get
$$
\begin{bmatrix} -x & -y & -1 & 0 & 0 & 0 & u*x & u*y & u \\ 0 & 0 & 0 & -x & -y & -1 & v*x & v*y & v \\ ... & ... & ... & ... & ... & ... & ... & ... & ...\end{bmatrix}*
\begin{bmatrix} h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & h_{33}\end{bmatrix} = \begin{bmatrix} 0 \\ ... \\ 0\end{bmatrix}
$$

https://youtu.be/hxbQ-F8u08U

In [78]:
def getH(cPoints, wPoints):
    X = wPoints[:,0]
    Y = wPoints[:,1]
    U = cPoints[:,0]
    V = cPoints[:,1]

    A = []

    for x,y,u,v in zip(X,Y,U,V):
        row1 = np.array([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        row2 = np.array([0, 0, 0, -x, -y, -1, v*x, v*y, v])

        A.append(row1)
        A.append(row2)

    A = np.array(A)

    # Singular Value Decomposition
    U, E, VT = np.linalg.svd(A, full_matrices=True)

    idx = np.argmin(E)

    # Last column of VT is the answer of H
    H = VT[idx].reshape((3,3))

    # H is defined up to a scale ==> set scale so that H[2,2] = 1
    H /= H[2,2]
    return H
    


In [79]:
wPoints = objpoints[0]
cPoints = imgpoints[0]
H = getH(cPoints, wPoints)

In [80]:
H

array([[1.19287339e+03, 6.67193126e+01, 7.50248921e+02],
       [8.22308400e+01, 1.18655205e+03, 7.64981679e+02],
       [9.68051574e-02, 1.12827181e-01, 1.00000000e+00]])

In [81]:
from scipy.optimize import curve_fit

Maximum Likelihood Estimation

https://www.researchgate.net/publication/303233579_Zhang%27s_Camera_Calibration_Algorithm_In-Depth_Tutorial_and_Implementation

In [82]:
def est_cPoints(wPoints, *H):
    """
    Value function for Levenberg-Marquardt refinement.
    """
    
    N = np.shape(wPoints)[0] // 2
    
    h11, h12, h13, h21, h22, h23, h31, h32, h33 = H

    X = wPoints[:N]
    Y = wPoints[N:]

    U = (h11 * X + h12 * Y + h13) / (h31 * X + h32 * Y + h33)
    V = (h21 * X + h22 * Y + h23) / (h31 * X + h32 * Y + h33)

    cPoints_new = np.concatenate((U,V))

    # print("cPoints_new shape", np.shape(cPoints_new))

    return cPoints_new

In [83]:
def J_refine(wPoints, *H):
    """
    Jacobian function for Levenberg-Marquardt refinement.
    """

    N = np.shape(wPoints)[0] // 2
    
    h11, h12, h13, h21, h22, h23, h31, h32, h33 = H

    X = wPoints[:N]
    Y = wPoints[N:]

    U = h11 * X + h12 * Y + h13
    V = h21 * X + h22 * Y + h23
    W = h31 * X + h32 * Y + h33

    row1 = np.stack((X/W, Y/W, 1.0/W, np.zeros_like(X), np.zeros_like(X), np.zeros_like(X), -U*X/W**2, -U*Y/W**2, -U/W**2)).T
    row2 = np.stack((np.zeros_like(X), np.zeros_like(X), np.zeros_like(X), X/W, Y/W, 1.0/W, -V*X/W**2, -V*Y/W**2, -V/W**2)).T

    J = np.concatenate((row1, row2))

    # print("J shape", np.shape(J))

    return J

In [84]:
def refine_H(wPoints, cPoints, H):    
    """
    Perform nonlinear least squares to refine linear homography estimates
    Args:
       H      : 3x3 homography matrix
       wPoints: Nx2 world frame planar model
       cPoints: Nx2 camera frame correspondences
    Returns:
       Refined 3x3 homography
    """
    wPoints = wPoints.ravel()
   #  print("wPoints shape", np.shape(wPoints))

    cPoints = cPoints.ravel()
   #  print("cPoints shape", np.shape(cPoints))

    H       = H.ravel()
   #  print("H shape", np.shape(H))

    popt, pcov = curve_fit(est_cPoints, wPoints, cPoints, p0=H, jac=J_refine)

    h_refined = popt

    # Normalize and reconstitute homography
    h_refined /= h_refined[-1]
    H_refined = h_refined.reshape((3,3))
   #  print(np.shape(H_refined))
    return H_refined

In [85]:
H = refine_H(wPoints, cPoints, H)

In [86]:
H

array([[ 9.63204708e+02,  2.67253051e+02,  7.59596296e+02],
       [-1.42977128e+02,  1.37069027e+03,  7.61158885e+02],
       [-1.01389900e-01,  3.02705305e-01,  1.00000000e+00]])

In [87]:
def getAllH(imgpoints, objpoints):
    all_H = []
    for wPoints in objpoints:
        for cPoints in imgpoints:
            H = getH(cPoints, wPoints)
            H = refine_H(wPoints, cPoints, H)
            all_H.append(H)
    return all_H

In [88]:
all_H = getAllH(imgpoints, objpoints)

/home/tvc/miniconda3/envs/ml/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [89]:
len(all_H)

100

Or,
$$
s \tilde{m} = H \tilde{M}
$$
with
$$
H = \begin{bmatrix} h_1 & h_2 & h_3\end{bmatrix} = \lambda K \begin{bmatrix} r_1 & r_2 & t\end{bmatrix} (1)
$$
So that,
$$
K^{-1}*h_1 = r_1 \\
K^{-1}*h_2 = r_2
$$
$$
\begin{cases}r_1, r_2 \text{ are orthonormal because it represents the rotation direction of each axis x } \& \text{ y} \\ r_{1}^{T}*r_{2}=0 \\ ||r_1|| = ||r_2|| = 1 \\
r_{1}^{T}*r_{1}=r_{2}^{T}*r_{2}=1\end{cases}
$$
We have,
$$
\begin{cases}
r_{1}^{T}*r_{2} = (K^{-1}*h_1)^{T} * (K^{-1}*h_2)=h_{1}^{T}*K^{-T}*K^{-1}*h_{2}\\
r_{1}^{T}*r_{1}-r_{2}^{T}*r_{2}=h_{1}^{T}*K^{-T}*K^{-1}*h_{1}-h_{2}^{T}*K^{-T}*K^{-1}*h_{2}=0
\end{cases} (2)
$$
Let call, $K^{-T}*K^{-1}=B$ and $B^{T}=B$ and $B = \begin{bmatrix} b_{11} & b_{12} & b_{13}\\b_{12} & b_{22} & b_{23}\\b_{13} & b_{23} & b_{33}\end{bmatrix}$

Note that, $B$ is symmetric, defined by a 6D vector
$$
b=\begin{bmatrix}B_{11} & B_{12} & B_{22} & B_{13} & B_{23} & B_{33}\end{bmatrix}^{T}
$$
Let the $i^{th}$ column of H be $h_i=\begin{bmatrix}h_{i1} & h_{i2} & h_{i3}\end{bmatrix}^{T}$. Then, we have
$$
h_{i}^{T}*B*h_{j}=v_{ij}^{T}*b
$$
with
$$
v_{ij}=\begin{bmatrix}h_{i1}*h_{j1} & h_{i1}*h_{j2}+h_{i2}*h_{j1} & h_{i2}*h_{j2} & h_{i3}*h_{j1}+h_{i1}*h_{j3} & h_{i2}*h_{j3}+h_{i3}*h_{j2} & h_{i3}*h_{j3}\end{bmatrix}^{T}
$$

In [90]:
def getVij(hi, hj):
    Vij = np.array([ hi[0]*hj[0], hi[0]*hj[1] + hi[1]*hj[0], hi[1]*hj[1], hi[2]*hj[0] + hi[0]*hj[2], hi[2]*hj[1] + hi[1]*hj[2], hi[2]*hj[2] ])
    return Vij.T

$(2)$ can be rewritten as with 1 image
$$
\begin{bmatrix}v_{12}^{T} \\ (v_{11} - v_{22})^{T}\end{bmatrix}*b=0
$$
If $n$ images, we get
$$
V*b=0
$$
by stack $n$ matrices like $(2)$-modified

In [91]:
def getV(all_H):
    # row1 = []
    # row2 = []
    V=[]
    for H in all_H:
        h1 = H[:,0]
        h2 = H[:,1]

        v12 = getVij(h1,h2)
        v11 = getVij(h1,h1)
        v22 = getVij(h2,h2)

        V.append(v12.T)
        V.append((v11-v22).T)

    V = np.array(V)

    # for H in all_H:
    #     h1 = H[:,0]
    #     h2 = H[:,1]

    #     v12 = getVij(h1,h2)
    #     v11 = getVij(h1,h1)
    #     v22 = getVij(h2,h2)

    #     row1.append(v12.T)
    #     row2.append((v11-v22).T)

    # row1, row2 = np.array(row1), np.array(row2)

    # # print(np.shape(row1))
    # # print(np.shape(row2))

    # V = np.concatenate((row1,row2))

    return V

In [92]:
def getB(all_H):
    V = getV(all_H)
    U, sigma, VT = np.linalg.svd(V)
    idx = np.argmin(sigma)
    b = VT[idx]

    B0, B1, B2, B3, B4, B5 = b

    # Form B = K_-T K_-1
    B = np.array([[B0, B1, B3],
                  [B1, B2, B4],
                  [B3, B4, B5]])

    return B

In [93]:
B = getB(all_H)

In [94]:
B

array([[-8.38362221e-06,  6.23900963e-07,  4.10561734e-03],
       [ 6.23900963e-07,  1.66727990e-06, -1.61867863e-03],
       [ 4.10561734e-03, -1.61867863e-03, -9.99990262e-01]])

We have, $K$ is a camera intrinsic matrix, and
$$
K=\begin{bmatrix} \alpha & \gamma & u_0 \\ 0 & \beta & v_0 \\ 0 & 0 & 1\end{bmatrix}
$$

Using Zhang's closed form solution for intrinsic parameters (Zhang, Appendix B, pg. 18)

$$
\begin{cases}
v_0 = (B_{12}B_{13}-B_{11}B_{23})/(B_{11}B_{22}-B_{12}^{2})\\
\lambda=B_{33}-[B_{13}^{2}+v_{0}(B_{12}B_{13}-B_{11}B_{23})]/B_{11}\\
\alpha=\sqrt{\lambda/B_{11}}\\
\beta=\sqrt{\lambda B_{11}/(B_{11}B_{22}-B_{12}^{2})}\\
\gamma=-B_{12} \alpha^{2} \beta / \lambda\\
u_0=\gamma v_0/\beta - B_{13} \alpha^{2}/\lambda
\end{cases}
$$

In [95]:
def getK(B):
    v0 = (B[0,1] * B[0,2] - B[0,0] * B[1,2]) / (B[0,0] * B[1,1] - B[0,1] * B[0,1])
    lambda_ = B[2,2] - (B[0,2] * B[0,2] + v0 * (B[0,1] * B[0,2] - B[0,0] * B[1,2])) / B[0,0]
    alpha = np.sqrt(lambda_ / B[0,0])
    beta = np.sqrt(lambda_ * B[0,0] / (B[0,0] * B[1,1] - B[0,1] * B[0,1]))
    gamma = -B[0,1] * alpha * alpha * beta / lambda_
    u0 = gamma * v0 / beta - B[0,2] * alpha * alpha / lambda_

    K = np.array([[alpha, gamma, u0],
                  [0.   , beta , v0],
                  [0.   , 0.   , 1.]])
    return K

In [96]:
K = getK(B)

/home/tvc/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in sqrt
  after removing the cwd from sys.path.


In [97]:
K

array([[         nan,          nan,          nan],
       [  0.        ,  50.68663101, 766.25718673],
       [  0.        ,   0.        ,   1.        ]])

From $(1)$,
$$
r_{1}=\lambda*A^{-1}*h_{1} \\
r_{2}=\lambda*A^{-1}*h_{2} \\
r_{3}=cross(r_{1},r_{2}) \\
t = \lambda*A^{-1}*h_{3}
$$
With
$$
\lambda = {1 \over ||A^{-1}*h_{1}||} = {1 \over ||A^{-1}*h_{2}||}
$$

In [98]:
def getR_n_T(K, all_H):
    all_RT = []
    for H in all_H:
        h1 = H[:,0]
        h2 = H[:,1]
        h3 = H[:,2]
        lamb = np.linalg.norm(np.dot(np.linalg.inv(K), h1), 2)

        r1 = np.dot(np.linalg.inv(K), h1) / lamb
        r2 = np.dot(np.linalg.inv(K), h2) / lamb
        r3 = np.cross(r1, r2)
        t = np.dot(np.linalg.inv(K), h3) / lamb
        RT = np.vstack((r1, r2, r3, t)).T
        all_RT.append(RT)

    return all_RT  

In [99]:
all_RT = getR_n_T(K, all_H)

In [100]:
all_RT

[array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, n

## Start the Calibration process!

In [101]:
def extractParamFromK(init_K, init_kc):
    alpha = init_K[0,0]
    gamma = init_K[0,1]
    beta = init_K[1,1]
    u0 = init_K[0,2]
    v0 = init_K[1,2]
    k1 = init_kc[0]
    k2 = init_kc[1]

    x0 = np.array([alpha, gamma, beta, u0, v0, k1, k2])
    return x0

In [102]:
def retrieveA(x0):
    alpha, gamma, beta, u0, v0, k1, k2 = x0
    A = np.array([[alpha, gamma, u0], [0, beta, v0], [0, 0, 1]]).reshape(3,3)
    kc = np.array([k1, k2]).reshape(2,1)
    return A, kc